In [1]:
!ls filimdb_evaluation/FILIMDB/

dev-b.labels  dev.labels  test-b.texts	train.labels  train_unlabeled.texts
dev-b.texts   dev.texts   test.texts	train.texts


In [2]:
#Загружаем текст
import os
import math
from collections import defaultdict
with open(f'filimdb_evaluation/FILIMDB/train.texts', 'r') as f:
    text = f.read()
reviews=[review for review in text.split('\n')]

with open(f'filimdb_evaluation/FILIMDB/train.labels', 'r') as f:
    text_labels = f.read()
labels=[label for label in text_labels.split('\n')]
#Посчитаем вероятности классов!!!
probability_pos=0
probability_neg=0

for i in labels:
    if i=='pos':
        probability_pos+=1

for i in labels:
    if i=='neg':
        probability_neg+=1
        
probability_pos=probability_pos/len(labels)
probability_neg=probability_neg/len(labels)
#Загрузим обучающую выборку в 2 списка позитивные и негативные отзывы.И сразу запихнём все отзывы одной метки в одну строку
reviews_pos=' '.join([reviews[i] for i in range(len(reviews)) if labels[i]=='pos'])#Хотел сделать маской красиво, но не получилось.Нужен Numpy
reviews_neg=' '.join([reviews[i] for i in range(len(reviews)) if labels[i]=='neg'])
#Может надо удалить бесполезный символ перевода строки тут? (<br /><br />)

In [3]:
def preprocessing(my_text):
    #Переводим в нижний регистр
    #Уберём все лишние символы(не буквы и цифры)
    return ''.join([symbol.lower() if symbol.isalnum() or symbol==' ' else ' '+symbol+' ' for symbol in my_text])
    

In [12]:
reviews_pos=preprocessing(reviews_pos)
reviews_neg=preprocessing(reviews_neg)

In [5]:
def tokenization(my_text):
    return [word for word in my_text.split(' ') if not word==' 'and not word=='  ' and not word=='']

In [6]:
reviews_pos=tokenization(reviews_pos)
reviews_neg=tokenization(reviews_neg)

In [7]:
#Создаём словари с вероятностями слова->вероятность
vocab_pos=defaultdict(float)
vocab_neg=defaultdict(float)
sum_pos=len(set(reviews_pos))+1
sum_neg=len(set(reviews_neg))+1
vocab_pos['<unk>']=1
vocab_neg['<unk>']=1
for w in reviews_pos:
    vocab_pos[w] += 1
for w in reviews_neg:
    vocab_neg[w] += 1
result_train=[vocab_neg,probability_neg,vocab_pos,probability_pos]

In [8]:
def classifier(text,params):#Классифицирует один отзыв и возращает метку
    text=tokenization(preprocessing(text))
    p_neg=0
    p_pos=0
      
    #####Bernoulli model####
    """
    text=set(text)
    for i in text:#прошлись по тем словам, что есть в тексте
        val=params[0].get(i,-1)
        if val==-1:
            p_neg+=math.log(params[0].get('<unk>'))
        else:
            p_neg+=math.log(val)
            
    for i,value in params[0].items():#прошлись по тем словам, коих нет в тексте
        if not i in text:
            try:
                p_neg+=math.log(1-value)
            except ValueError:
                p_neg+=0
    

    p_neg*=params[1]
    
    for i in text:
        val=params[2].get(i,-1)
        if val==-1:
            p_pos+=math.log(params[2].get('<unk>'))
        else:
            p_pos+=math.log(val)
           
    for i,value in params[2].items():#прошлись по тем словам, коих нет в тексте
        if not i in text:
            try:
                p_pos+=math.log(1-value)
            except ValueError:
                p_neg+=0
    
    
    p_pos*=params[3]
    
    """
    #####Multinomial model####
    text_set=set(text)
    
    for i in text_set:
        val=params[0].get(i,-1)
        if val==-1:
            p_neg+=math.log(params[0].get('<unk>'))*text.count(i)
        else:
            p_neg+=math.log(val)*text.count(i)
    p_neg*=params[1]
    
    for i in text_set:
        val=params[2].get(i,-1)
        if val==-1:
            p_neg+=math.log(params[2].get('<unk>'))*text.count(i)
        else:
            p_neg+=math.log(val)*text.count(i)
    p_pos*=params[3]
    
    ##########################
    return 'pos' if p_pos>p_neg else 'neg'

In [9]:
def classify(texts, params):
    reviews=[review for review in text.split('\n')]
    return [classifier(i,params) for i in reviews]

In [10]:
res=classify(text,result_train)

In [11]:
accuracy=0
for i in range(len(labels)):
    if res[i]==labels[i]:
        accuracy+=1
print(accuracy/len(text_labels))

0.12466666666666666
